In [ ]:
!nvidia-smi

In [ ]:
#!pip install transformers accelerate peft datasets bitsandbytes sentencepiece

In [ ]:
## Importing necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset, Dataset
import pandas as pd
import torch

In [ ]:
# loading the final dataset
## for reproduction purposes, we are using the final cleaned dataset directly from the github repo created for the dataset
df = pd.read_csv(
    "https://raw.githubusercontent.com/nyarderr/moodmate-data/refs/heads/main/goemotions_final.csv"
)
df.head()

In [ ]:
## Convert the dataset to Hugging Face Dataset format
def to_hf_dataset(row):
    return {
        "text": row["text"],
        "labels": row["final_emotion"],
    }

dataset = df.apply(to_hf_dataset, axis=1, result_type="expand")
hf_dataset = Dataset.from_pandas(dataset)
hf_dataset = hf_dataset.train_test_split(test_size=0.1, seed=42)
hf_dataset

In [ ]:
## Load Qwen tokenizer and model
#model_name = "Qwen/Qwen1.5-1.8B"
model_name = "Qwen/Qwen1.5-0.5B"

## Loading the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

## 4-bit quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

## Loading the model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    quantization_config=bnb_config,
    # load_in_8bit=True,
    device_map="auto",
    # llm_int8_enable_fp32_cpu_offload=True,
    # offload_folder="../models/offload"
)

## Preparing the model for k-bit training
model = prepare_model_for_kbit_training(model)

In [ ]:
### define tokenizer function
def tokenize(batch):
    ## full training string
    prompt = (f"Instrcution: Identify the emotion of the following text.\n"
              f"Text:{batch['text']}\n"
              f"Emotion:")

    full_text = f"{prompt} {batch['labels']}"


    ## tokenizing full text
    tokenized = tokenizer(full_text, padding='max_length', truncation=True, max_length=256)

    ## labels only
    labels = tokenized["input_ids"].copy()

    ## getting prompt token ids
    prompt_tokens = tokenizer(prompt, padding='max_length', truncation=True, max_length=256)["input_ids"]

    ## masking labels
    for i in range(len(prompt_tokens)):
        if prompt_tokens[i] != tokenizer.pad_token_id:
            labels[i] = -100
    
    tokenized["labels"] = labels
    return tokenized

## Applying the tokenization function to the dataset
tokenized_dataset = hf_dataset.map(tokenize, batched=False)
tokenized_dataset

In [ ]:
## Applying LoRA to the model
#LoraConfig?
lora_config = LoraConfig(
    r=8, # rank(number of wheels)
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1, # regularization
    bias="none",
    task_type="CAUSAL_LM" # casual language modeling task
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
### choose max steps to epochs
#batch_size = 4*4  # per_device_train_batch_size * gradient_accumulation_steps
#num_samples = len(tokenized_dataset['train'])
#steps_per_epoch = num_samples // batch_size
#steps_per_epoch # 3095 steps per epoch

In [ ]:
## Training arguments
training_args = TrainingArguments(
    output_dir="../models/qwen-lora-goemotions",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    warmup_steps=50,
    max_steps=3095,
    learning_rate=2e-4,
    fp16=True,  # half precision training
    logging_steps=20,
    report_to="wandb",
    save_steps=200,
    save_total_limit=2,
)

In [ ]:
## save the fine-tuned model
trainer.save_model("../models/qwen-lora-goemotions")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
)

trainer.train(resume_from_checkpoint=True)